In [ ]:
# The code is importing various modules and helper functions from different files.

import re
import time

from bs4 import BeautifulSoup, NavigableString, ResultSet, Tag

import rokomari_scrap_helper as rsh
from helpers import common_helper as ch
from helpers import json_helper as jh
from helpers import scraping_helper as sh
from models.book import Book
from models.book_category import BookCategory

In [ ]:
book_category_list: list[BookCategory] = rsh.get_book_category_list()

print("Total Book Categories Count: ", book_category_list.__len__())

dynamic_book_category_list = rsh.get_book_categories_containing_url_segment(
    book_category_list, "/book/category"
)
print("Total Dynamic Book Categories Count: ", dynamic_book_category_list.__len__())

static_book_category_list = rsh.get_book_categories_not_containing_url_segment(
    book_category_list, "/book/category"
)
print("Total Static Book Categories Count: ", static_book_category_list.__len__())

In [ ]:
book_category = dynamic_book_category_list[0]

pagination_response = sh.get_http_response(book_category.url)

book_list: list[Book] = []

if pagination_response.status_code != 200:
    book_list = []
    
else:
    pagination_soup = sh.parse_html_content_as_string(pagination_response.text)

In [ ]:
pagination_element = sh.find_one_by_class_name(pagination_soup, 'div', 'pagination')

pagination_list = pagination_element.text.strip().split()
numbers = [int(x) for x in pagination_list if x.isdigit()]
largest_page_number = max(numbers)
print(f"Total Page Numbers: {largest_page_number}")

In [ ]:
list_page_url = f"{book_category.url}&page={largest_page_number}"
book_list_page_response = sh.get_http_response(book_category.url)

book_card_list: list[Tag] = []
book_list: list[Book] = []

if book_list_page_response.status_code != 200:
    book_list = []
else:
    book_list_page_soup = sh.parse_html_content_as_string(book_list_page_response.text)

    book_card_list: list[Tag] = sh.find_all_by_class_name(
        book_list_page_soup, "div", "books-wrapper__item"
    )

In [ ]:
print(book_card_list.__len__())

for book_card_item in book_card_list:
    
    # book_url
    tag_item = sh.find_all_by_tag_name(book_card_item, "a")[1]
    all_book_urls = sh.get_value_of_attributes([tag_item], "href")
    book_url = rsh.base_url + all_book_urls[0].get("value").strip()
    
    # image_url
    book_image_item = sh.find_all_by_tag_name(book_card_item, "img")[1]
    all_image_urls = sh.get_value_of_attributes([book_image_item], "src")
    image_url = rsh.base_url + all_image_urls[0].get("value").strip()
    
    
    # book-title
    book_title_item = sh.find_one_by_class_name(book_card_item, "h4", "book-title")
    book_title = book_title_item.text.strip()
    
    # book-author
    book_author_item = sh.find_one_by_class_name(book_card_item, "p", "book-author")
    book_author = book_author_item.text.strip()
    
    
    
    
    
    book_list.append(
        Book(
            title=book_title,
            author=book_author,
            isAvailable=False,
            originalPrice=0,
            currentPrice=0,
            imageUrl=image_url,
            bookUrl=book_url,
            category=book_category,
        )
    )
    
print(jh.data_to_json_string(book_list))
